# 51515 Medium Angular Motors System Identification

The following notebook handles with system identification of the 51515, Lego Mindstorm, Robot Inventor, Medium Angular Motors.

#### Scope

The motivation behind this notebook is to understand the relationship between the applied [duty cycle](https://docs.pybricks.com/en/latest/pupdevices/motor.html#pybricks.pupdevices.Motor.dc) through the [pybricks](https://pybricks.com/) firmware and the force applied to a idealized cart in the [idealized inverted pendulum robot model](https://ctms.engin.umich.edu/CTMS/index.php?example=InvertedPendulum&section=SystemModeling). 

## Motor characteristics

In order to establish the relationship between the applied [duty cycle](https://docs.pybricks.com/en/latest/pupdevices/motor.html#pybricks.pupdevices.Motor.dc) and the force applied to the idealized cart, the following motor characteristics is desired to be established and understood: 

* Internal resistance       ($R$)
* Torque constant           ($k_t$)
* Speed-torque curve
* Static friction torque    ($\tau_{fric}$)
* Encoder accuracy and delay
* Duty cycle and current relationship

#### Internal resistance

As prensented in [University of Utah MEC lab excercise](https://my.mech.utah.edu/~me3200/labs/S04Labs/S04_MotorChar_L04.pdf), during steady state, the electrical equation of a DC motor simplifies to:

\begin{equation}
V = I R + K_e \omega 
\end{equation}

Combining this with [measured data](https://www.philohome.com/motors/motorcomp.htm) it is possible to use linear regression for estimating $K_e$ and $I$.



In [3]:
import statsmodels.api as sm
from conversions import rpm_to_rad_per_s
import numpy as np

omega = rpm_to_rad_per_s(rpm=np.array([24.0, 63.0, 105.0, 138.0, 180.0, 213.0])).reshape((6,1))

I = np.array([0.29, 0.28, 0.29, 0.30, 0.31, 0.32]).reshape((6,1))
V = np.array([4.5, 6.0, 7.5, 9.0, 10.5, 12.0])

X = np.hstack((I, omega))

y = V

print(X.shape)
print(y.shape)

results = sm.OLS(y, X).fit()
print(results.summary())





(6, 2)
(6,)
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          2.152e+04
Date:                Sat, 11 Feb 2023   Prob (F-statistic):                    8.64e-09
Time:                        10:05:01   Log-Likelihood:                          6.3995
No. Observations:                   6   AIC:                                     -8.799
Df Residuals:                       4   BIC:                                     -9.215
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------

c:\repos\51515-System-Identification\51515-System-Identification\.venv\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


#### Intenal resistance

As prensented in the video [Northwestern Robotics youtube](https://www.youtube.com/watch?v=pxtRlKs0pAg) and [BSc Thesis](https://www.diva-portal.org/smash/get/diva2:916184/FULLTEXT01.pdf), during steady state operation the DC motor curcuit can be modelled as:

\begin{equation}
V = k_t \omega + I R 
\end{equation}

This can be utilized for estimated the resistance in the motor, by running an test with duty cycle at zero, hence zero angular velocity, as follows:





In [ ]:
from data import *

# Loading and visualizing data:
data = load_and_format_data("motor_test_04_02_2023__no1.txt")
visualize_data(data)

# Select and verifying subset of data
data = select_data_subset(data, [0,1950])
visualize_data(data)

# Estimating current and voltage
I, V = estimate_average_voltage_and_current(data)
print(f"Estimated current: {np.round(I,3)} [mA];\tEstimated voltage: {np.round(V,3)} [mV]")
print(f"Used for calculating resistance: {np.round(V/I,3)}")

#### Torque constant

Given that the torque constant is defined as

\begin{equation}
\tau =  k_t I \qquad \Leftrightarrow \qquad k_t = \dfrac{\tau}{I}
\end{equation}

equation (1) can be rewritten as

\begin{equation}
\omega = \dfrac{V}{k_t}  -  \dfrac{R \tau}{k_t^2}     
\end{equation}

which can be used for plotting the speed-torque curve.


Using 

Next steps:
* Estimate R from measurements with duty_cycle = 0 %.
* Estimtage the torque constant
* Plot the speed-torque curve and compare to measurents from [Lego Motors Compared Characteristics](https://www.philohome.com/motors/motorcomp.htm).
* Run experiments on different motors to see if they are the same.

The Robot Inventor, Medium Angular Motors are equivalent to the Spike Prime Medium Angular Motors. The following references are good starting points

* [Lego Offical Technical Specification](https://le-www-live-s.legocdn.com/sc/media/files/support/spike-prime/techspecs_technicmediumangularmotor-19684ffc443792280359ef217512a1d1.pdf)
* [Lego Motors Compared Characteristics](https://www.philohome.com/motors/motorcomp.htm)